In [21]:
import pickle
import pandas as pd

# Carregando o modelo
with open('../model/modelo_treinado.pkl', 'rb') as f:
    modelo = pickle.load(f)

# Colunas utilizadas no treinamento
colunas_modelo = ['Genero', 'Idade', 'Nivel_Academico', 'Tempo_Tela_h_dia',
                   'Duracao_Sono_h_dia','Atividade_Fisica_h_semana', 
                   'Ansiedade_Antes_Provas', 'Mudanca_Desempenho_Academico']

In [ ]:
# Mapeamentos baseados no tratamento com LabelEncoder
mapeamento_genero = {'Female': 0, 'Male': 1, 'Other': 2}
mapeamento_genero_interface = {'Feminino': 0, 'Masculino': 1, 'Outro': 2}

mapeamento_nivel = {
    'BA': 0, 'BSc': 1, 'BTech': 2,
    'Class 10': 3, 'Class 11': 4, 'Class 12': 5,
    'Class 8': 6, 'Class 9': 7,
    'MA': 8, 'MSc': 9, 'MTech': 10
}
mapeamento_nivel_interface = {
    'Graduação - BA': 0, 'Graduação - BSc': 1, 'Graduação - BTech': 2,
    'Ensino Fundamental (8ª série)': 6, 'Ensino Fundamental (9ª série)': 7,
    'Ensino Médio (1º ano)': 4, 'Ensino Médio (2º ano)': 5, 'Ensino Médio (3º ano)': 3,
    'Pós - MA': 8, 'Pós - MSc': 9, 'Pós - MTech': 10
}

mapeamento_ansiedade = {'Sim': 1, 'Não': 0}
mapeamento_desempenho = {'Piorou': 0, 'Melhorou': 1, 'Manteve-se': 2}
mapeamento_saida = {0: 'Alto', 1: 'Baixo', 2: 'Médio'}

# Função de predição
def nivel_estresse(Genero, Idade, Nivel_Academico, Tempo_Tela_h_dia,
                   Duracao_Sono_h_dia, Atividade_Fisica_h_semana,
                   Ansiedade_Antes_Provas, Mudanca_Desempenho_Academico):

    if Ansiedade_Antes_Provas not in mapeamento_ansiedade:
        return "Erro: selecione se sente ansiedade antes de provas."

    if Mudanca_Desempenho_Academico not in mapeamento_desempenho:
        return "Erro: selecione a mudança no desempenho acadêmico."
    
    dados = pd.DataFrame([[
        mapeamento_genero_interface[Genero],
        Idade,
        mapeamento_nivel_interface[Nivel_Academico],
        Tempo_Tela_h_dia,
        Duracao_Sono_h_dia,
        Atividade_Fisica_h_semana,
        mapeamento_ansiedade[Ansiedade_Antes_Provas],
        mapeamento_desempenho[Mudanca_Desempenho_Academico]
    ]], columns=colunas_modelo)

    previsao = modelo.predict(dados)[0]
    return f"Nível de estresse previsto: {mapeamento_saida[previsao]}"

In [23]:
import gradio as gr # Importa a biblioteca Gradio para criar interfaces gráficas simples em Python

# --- Interface Gradio ---
opcoes_genero = list(mapeamento_genero_interface.keys())
opcoes_nivel = list(mapeamento_nivel_interface.keys())
opcoes_ansiedade = ['Sim', 'Não']
opcoes_desempenho = ['Melhorou', 'Piorou', 'Manteve-se']
Ansiedade = gr.Radio(opcoes_ansiedade, label="Sente ansiedade antes de provas?", value="Não")


with gr.Blocks() as app:
    gr.Markdown("### Previsão de Nível de Estresse")

    with gr.Row():
        Genero = gr.Dropdown(opcoes_genero, label="Gênero")
        Idade = gr.Number(label="Idade", value=18)

    with gr.Row():
        Nivel_Academico = gr.Dropdown(opcoes_nivel, label="Nível Acadêmico")
        Tempo_Tela = gr.Slider(0, 16, step=0.5, label="Tempo de tela (horas/dia)")

    with gr.Row():
        Sono = gr.Slider(0, 12, step=0.5, label="Duração do sono (horas/dia)")
        Atividade = gr.Slider(0, 20, step=1, label="Atividade física (horas/semana)")

    with gr.Row():
        Ansiedade = gr.Radio(opcoes_ansiedade, label="Sente ansiedade antes de provas?")
        Desempenho = gr.Dropdown(opcoes_desempenho, label="Mudança no desempenho acadêmico")

    botao = gr.Button("Prever")
    saida = gr.Label(label="Resultado")

    botao.click(
        fn=nivel_estresse,
        inputs=[Genero, Idade, Nivel_Academico, Tempo_Tela, Sono, Atividade, Ansiedade, Desempenho],
        outputs=saida
    )

app.launch(share=True)

* Running on local URL:  http://127.0.0.1:7868

Could not create share link. Missing file: C:\Users\caiob\.cache\huggingface\gradio\frpc\frpc_windows_amd64_v0.3. 

Please check your internet connection. This can happen if your antivirus software blocks the download of this file. You can install manually by following these steps: 

1. Download this file: https://cdn-media.huggingface.co/frpc-gradio-0.3/frpc_windows_amd64.exe
2. Rename the downloaded file to: frpc_windows_amd64_v0.3
3. Move the file to this location: C:\Users\caiob\.cache\huggingface\gradio\frpc


In [25]:
import pandas as pd

# Caminho para o local do arquivo de dados
df = pd.read_csv('../datasets/dados_modificados.csv')

# Distribuição dos dados
print(df['Nivel_Estresse'].value_counts())

Nivel_Estresse
2    492
1    327
0    181
Name: count, dtype: int64


O modelo só retorna o nível de estresse médio.